In [25]:
import psycopg2
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import numpy as np

%matplotlib inline
sns.set(style="white")

In [26]:
## connect to database
con = psycopg2.connect("dbname='doctordb' user='cathy'")

In [27]:
## excluded providers that are in summary
## (join on leie.NPI=summary.npi)
## select exclusion dates > 2015-01-01 so that payment claims reflect full 2014 year
q = """SELECT summary.npi
            , summary.provider_type
            , summary.nppes_provider_state
            , leie.EXCLDATE as exclusion_date
            FROM summary
                JOIN leie 
                ON (leie.NPI = summary.npi)
            WHERE leie.EXCLDATE > '2015-01-01';"""
providers = pd.read_sql_query(q, con=con)

In [28]:
providers.shape

(199, 4)

In [29]:
providers.head()

,npi,provider_type,nppes_provider_state,exclusion_date
0,1972607398,Internal Medicine,FL,2016-09-20
1,1487765467,Clinical Psychologist,OH,2016-09-20
2,1861487308,Family Practice,FL,2016-08-18
3,1811282098,Internal Medicine,TX,2016-08-18
4,1689986945,Occupational therapist,NY,2016-08-18


In [31]:
## how many providers per provider type?
(providers.groupby('provider_type')['npi'].
        count().
        reset_index().
        rename(columns={'npi':'count'}).
        sort_values('count', ascending=False))

,provider_type,count
16,Internal Medicine,45
10,Family Practice,25
38,Psychiatry,20
12,General Practice,12
5,Chiropractic,8
23,Nurse Practitioner,8
36,Podiatry,7
33,Physical Medicine and Rehabilitation,7
8,Dermatology,6
35,Physician Assistant,5


In [32]:
## how many providers per state?
(providers.groupby('nppes_provider_state')['npi'].
        count().
        reset_index().
        rename(columns={'npi':'count'}).
        sort_values('count', ascending=False))

,nppes_provider_state,count
10,IL,25
2,CA,20
6,FL,14
22,NY,13
16,MI,13
19,NJ,13
28,TX,8
14,MD,7
27,TN,7
11,IN,7


In [47]:
providers[providers.provider_type=='Orthopedic Surgery']

,npi,provider_type,nppes_provider_state,exclusion_date
35,1174693949,Orthopedic Surgery,NY,2016-06-20
153,1427029925,Orthopedic Surgery,MI,2015-07-20
193,1316151525,Orthopedic Surgery,NY,2015-02-19


In [39]:
providers[providers.provider_type=='Medical Oncology']

,npi,provider_type,nppes_provider_state,exclusion_date
151,1093779456,Medical Oncology,TN,2015-08-20


In [40]:
providers[providers.provider_type=='Hematology/Oncology']

,npi,provider_type,nppes_provider_state,exclusion_date
103,1871596098,Hematology/Oncology,FL,2015-12-20
162,1285673012,Hematology/Oncology,TX,2015-06-18


In [42]:
providers[providers.provider_type=='Internal Medicine'].head()

,npi,provider_type,nppes_provider_state,exclusion_date
0,1972607398,Internal Medicine,FL,2016-09-20
3,1811282098,Internal Medicine,TX,2016-08-18
8,1528051307,Internal Medicine,PA,2016-08-18
23,1225082886,Internal Medicine,PA,2016-07-20
29,1689756827,Internal Medicine,NY,2016-06-20


## `provider_type` = Internal Medicine has the most cases of fraud (45)
How many of these internal medicine providers have claims data (i.e. in payments table) as well?

In [53]:
internal_med_npi = tuple(providers.loc[providers.provider_type=='Internal Medicine','npi'])

In [54]:
internal_med_npi[:5]

('1972607398', '1811282098', '1528051307', '1225082886', '1689756827')

In [62]:
## excluded providers that are in summary
## (join on leie.NPI=summary.npi)
## select exclusion dates > 2015-01-01 so that payment claims reflect full 2014 year
q = """SELECT * FROM payments WHERE npi IN {0};""".format(internal_med_npi)
payments = pd.read_sql_query(q, con=con)

In [63]:
payments.shape

(527, 26)

In [65]:
payments.drop_duplicates(['npi']).shape

(44, 26)

44 internal medicine providers made claims in payments table during 2014.

1 fraudulent provider is in the summary table but not in payments.

In [67]:
## number of rows / distinct hcpcs_code claims made per provider
payments.groupby('npi')['npi'].count()

npi
1043238421    42
1043257744    21
1053458018    10
1073808945     5
1093851164    13
1124016480    29
1134139884    14
1174543342     8
1184786196     2
1225011612     8
1225082886    13
1235133794     5
1245253012     1
1245279645     7
1306830021     9
1316935406    15
1336203603    21
1396906160    12
1417967647     8
1427071059     8
1427090562    30
1427098268    12
1497737266     2
1528051307     2
1528111341    13
1558364166     9
1558405647     3
1568410496    21
1568494169     1
1578615316     9
1588694343    31
1609829985    21
1679549513     2
1689756827     2
1770656506    15
1811282098     3
1831121979     9
1841206307     8
1871571406    36
1891722237    12
1912032780     2
1922032770    13
1962461640     3
1972607398    17
Name: npi, dtype: int64

In [71]:
payments.drop_duplicates('npi')[['npi','nppes_provider_last_org_name','provider_type']]

,npi,nppes_provider_last_org_name,provider_type
0,1043238421,HUSSAIN,Internal Medicine
42,1043257744,DE LUCA,Internal Medicine
63,1053458018,APONTE,Internal Medicine
73,1073808945,HOGAN,Internal Medicine
78,1093851164,LUCERO,Internal Medicine
91,1124016480,HAMER,Internal Medicine
120,1134139884,BRUCE,Internal Medicine
134,1174543342,WHITEUS,Internal Medicine
142,1184786196,JAKIMIEC,Internal Medicine
144,1225011612,NDOLO,Internal Medicine


In [76]:
## write out to csv the npi s of the 44 internal medicine npi fraudsters
(pd.DataFrame(payments.drop_duplicates('npi')['npi'])
.to_csv('internal_medicine_npi_fraud.csv', index=False))